In [1]:
import torch
import torch.nn as nn

import pandas as pd
import pyarrow as pa
from datasets import Dataset

from model import BertForTokenClassification
from utils.NERutils import readDataset , getVocabFeatures, findMaxLength, encodeDataFrame

from transformers import AutoConfig, AutoTokenizer


In [2]:
train_path = "data/train.parquet"
dev_path = "data/dev.parquet"
test_path = "data/test.parquet"

train_dataset = readDataset(train_path)
dev_dataset = readDataset(dev_path)
test_dataset = readDataset(test_path)

In [3]:
tags, index2tag, tag2index = getVocabFeatures(train_dataset)

In [4]:
device = torch.device('cpu')

In [5]:
# Config
bert_model_name = "bert-base-multilingual-cased"
bert_config = AutoConfig.from_pretrained(
    bert_model_name, 
    num_labels=len(tags), 
    id2label=index2tag, 
    label2id=tag2index
)

model = BertForTokenClassification.from_pretrained(bert_model_name, config=bert_config, tags=tags).to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Load model
model.load_state_dict(torch.load("test_model", map_location=device))

<All keys matched successfully>

In [7]:
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

In [11]:
# Tokenizer
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# Max lenght
max_length = findMaxLength(df=test_dataset, tokenizer=bert_tokenizer)

# Filter
filter=test_dataset['dagw_domain']=='Legal'

test_dataset_encoded = encodeDataFrame(test_dataset, bert_tokenizer, tag2index, max_length, filter)

In [13]:
model(test_dataset_encoded)

InvalidIndexError: (slice(None, None, None), [-1, 0])